In [2]:
import os
import warnings
import tifffile
import datetime
import sys
sys.path.append("..")

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.ndimage import uniform_filter1d
from scipy.optimize import curve_fit

import obf_support

# warning suppression
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

def gaussian2D(xy, amplitude, xo, yo, sigma, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma**2) + (np.sin(theta)**2)/(2*sigma**2)
    b = -(np.sin(2*theta))/(4*sigma**2) + (np.sin(2*theta))/(4*sigma**2)
    c = (np.sin(theta)**2)/(2*sigma**2) + (np.cos(theta)**2)/(2*sigma**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

c:\Users\alvelidjonatan\miniconda3\envs\mfxdata\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
topfolders = [os.path.join(parentdir, 'exampledata\\gag\\241114'),
              os.path.join(parentdir, 'exampledata\\gag\\250117')]
eventfolders_all = []
for topfolder in topfolders:
    samplefolders = os.listdir(topfolder)
    for samplefolder in samplefolders:
        if 'confocal' not in samplefolder:
            if os.path.isdir(os.path.join(topfolder, samplefolder)):
                eventfolders = os.listdir(os.path.join(topfolder, samplefolder))
                for eventfolder in eventfolders:
                    if os.path.isdir(os.path.join(topfolder, samplefolder, eventfolder)):
                        if 'nomfx' not in eventfolder:
                            if 'manual' not in eventfolder:
                                if any(filename.endswith('.npy') for filename in os.listdir(os.path.join(topfolder, samplefolder, eventfolder))):
                                    eventfolders_all.append(os.path.join(topfolder, samplefolder, eventfolder))
print(eventfolders_all)

['c:\\Users\\alvelidjonatan\\Documents\\GitHub\\etMINFLUX-analysis-public\\exampledata\\gag\\241114\\sample3\\e2', 'c:\\Users\\alvelidjonatan\\Documents\\GitHub\\etMINFLUX-analysis-public\\exampledata\\gag\\250117\\sample2\\e1']


In [ ]:
## SINGLE ROI FOLLOW

%matplotlib agg
%matplotlib agg

eventfolders = eventfolders_all
for folder in eventfolders:
    print(folder)
    filelist = os.listdir(folder)
    filelist_npy = [file for file in filelist if file.endswith('.npy')]
    filelist_confs = [file for file in filelist if 'conf' in file and 'analysis' not in file and 'stack' not in file]
    filelist_msr = [file for file in filelist if file.endswith('.msr')]
    filelist_logs = [file for file in filelist if file.endswith('.txt')]

    # load msr file to read confocal image settings
    # get metadata from confocal image in msr file (pixel size, image shape, image size, origin offset)
    file_msr = os.path.join(folder, filelist_msr[0])
    msr_dataset = obf_support.File(file_msr)
    conf_msr_stack_index = 0  # in currently used imspector template file, the confocal dataset is always stack 0 in the .msr file. This might change with other templates used.
    conf_stack = msr_dataset.stacks[conf_msr_stack_index]
    conf_img_last = [np.reshape(conf_stack.data, (conf_stack.shape[0], conf_stack.shape[1])).T]  # read last confocal image from msr file
    pxsize = conf_stack.pixel_sizes[0]*1e6
    pxshift = pxsize/2
    conf_size_px = (conf_stack.shape[0], conf_stack.shape[1])
    conf_size = (conf_stack.lengths[0]*1e6, conf_stack.lengths[1]*1e6)
    conf_offset = (conf_stack.offsets[0]*1e6, conf_stack.offsets[1]*1e6)
    # get roi pos
    roi_pos = (int(filelist_npy[0].split('[')[1].split(',')[0]),int(filelist_npy[0].split(']')[0].split(',')[1]))
    roi_pos_um = (roi_pos[0]*pxsize+conf_offset[0], roi_pos[1]*pxsize+conf_offset[1])

    # set other parameters for the plotting
    N_rois = len(filelist_msr)
    roi_idx_cum = 0
    nl = '\n'
    plt_roi_size = int(np.ceil(0.8/pxsize))
    spot_sum_size = int(np.ceil(0.24/pxsize))
    mfx_size = 0.8
    large_sum_size = int(np.ceil(mfx_size*1.2/2/pxsize))
    t_finframe = 3  # time in s for final confocal frame preevent and the time between that frame and mfx initialization (should always be 2-5 s, depending on confocal frame size)
    t_mfxtoconf = 9  # time in s between a mfx cycle end and a confocal frame start (+7s due to DM error)

    # get all confocal images in one stack
    images_conf = []
    for conf_file in filelist_confs:
        images_conf.append(tifffile.imread(os.path.join(folder, conf_file)))
    frames_preevent = len(images_conf[0])
    images_conf = np.concatenate(images_conf)
    images_conf = np.concatenate([images_conf, conf_img_last])

    # read timing info from log file
    file_log = os.path.join(folder, filelist_logs[0])
    log_dict = {}
    with open(file_log) as f:
        for line in f:
            key, sep, value = line.strip().partition(": ")
            log_dict[key] = value
    t_mfxcycs = []
    try:
        t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate-cycle0'], '%Hh%Mm%Ss%fus')
        t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end-cycle0'], '%Hh%Mm%Ss%fus')
        try:
            t_mfxcyc1 = datetime.datetime.strptime(log_dict['mfx_end-cycle1'], '%Hh%Mm%Ss%fus')
        except:
            t_mfxcyc1 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
        t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between confocal frames during MFX
    except:
        try:
            t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate-cycle0'], '%Hh%Mm%Ss%fus')
        except:
            t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate'], '%Hh%Mm%Ss%fus')
        t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
        t_sec_mfx = (t_mfxcyc0-t_confevent_end).seconds  # time between confocal frames during MFX
    try:
        t_conf0_start = datetime.datetime.strptime(log_dict['experiment_start'], '%Hh%Mm%Ss%fus')
    except:
        t_conf0_start = datetime.datetime.strptime('12h00m00s000000us', '%Hh%Mm%Ss%fus')
    t_sec_frames_preevent = (t_confevent_end-t_conf0_start).seconds  # time between confocal frames before MFX
    t_sec_mfx0 = (t_mfxcyc0-t_confevent_end).seconds  # time for one MFX rec

    # get t array for all confocal frames
    t = np.linspace(0, t_sec_frames_preevent-t_finframe, frames_preevent)
    t_eventdetect = t[-1]
    t = np.append(t, t[-1]+t_sec_mfx0+t_mfxtoconf)
    for _ in range(len(filelist_confs)-2+1):
        t = np.append(t,t[-1]+t_sec_mfx+t_mfxtoconf)

    # create figure for plotting
    cols = 4
    rows = 1+(len(images_conf)//cols+1)  # 1 for plots, the rest for the number of confocal frames
    fig, ax = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
    # get intensity limits for confocal images
    conf_zoom_lim = (np.min(images_conf[:,roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size]), np.max(images_conf[:,roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size]))
    # get x,y lims for confocal image zooms
    lims_confimg = np.array([roi_pos_um[0], roi_pos_um[0], roi_pos_um[1], roi_pos_um[1]]) + np.array([-plt_roi_size, plt_roi_size, -plt_roi_size, plt_roi_size])*pxsize
    # plot image zooms and plots of intensity over time
    spot_sums = []
    large_sums = []
    dxs = []
    dys = []
    dxerrs = []
    dyerrs = []
    for conf_idx_p in range(len(images_conf)):
        # summing
        conf_idx = len(images_conf)-conf_idx_p-1
        plotrow = conf_idx//cols+1
        plotcol = conf_idx%cols
        conf_img = images_conf[conf_idx_p]
        conf_img_spot_cut = conf_img[roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size+1, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size+1]
        conf_img_large_cut = conf_img[roi_pos[1]-large_sum_size:roi_pos[1]+large_sum_size+1, roi_pos[0]-large_sum_size:roi_pos[0]+large_sum_size+1]
        spot_sums.append(np.sum(np.sum(conf_img_spot_cut))/spot_sum_size**2)
        large_sums.append(np.sum(np.sum(conf_img_large_cut))/large_sum_size**2)
        rolling_window = 9
        spot_sums_sm = uniform_filter1d(spot_sums, size=rolling_window, mode='nearest')
        large_sums_sm = uniform_filter1d(large_sums, size=rolling_window, mode='nearest')
        # fitting centroid of spot with symmetric 2D gaussian
        x = np.linspace(0, 2*(large_sum_size+1)-2, 2*(large_sum_size+1)-1)
        y = np.linspace(0, 2*(large_sum_size+1)-2, 2*(large_sum_size+1)-1)
        x, y = np.meshgrid(x, y)
        try:
            initial_guess = (np.max(conf_img_large_cut), large_sum_size, large_sum_size, 1, 0, 10)
            popt, pcov = curve_fit(gaussian2D, (x, y), conf_img_large_cut.ravel(), p0=initial_guess)
            # get offsets from pixel-position
            dx = (popt[1]-large_sum_size)*pxsize
            dy = (popt[2]-large_sum_size)*pxsize
            x0err = np.sqrt(np.diag(pcov))[2]
            y0err = np.sqrt(np.diag(pcov))[1]
        except:
            dx = np.nan
            dy = np.nan
            x0err = np.nan
            y0err = np.nan
        dxs.append(dx)
        dys.append(dy)
        dxerrs.append(x0err)
        dyerrs.append(y0err)
        # plotting
        img_onlyconf = ax[plotrow,plotcol].imshow(conf_img, cmap='hot', vmin=conf_zoom_lim[0], vmax=conf_zoom_lim[1])
        if conf_idx_p == 0:
            extents_confimg = np.array(img_onlyconf.get_extent())*pxsize+[pxsize/2, pxsize/2, pxsize/2, pxsize/2]+[conf_offset[0], conf_offset[0], conf_offset[1], conf_offset[1]]
        img_onlyconf.set_extent(extents_confimg)  # scale overlay image to the correct pixel size for the tracks
        ax[plotrow,plotcol].set_xlim(lims_confimg[:2])
        ax[plotrow,plotcol].set_ylim(lims_confimg[2:])
        if conf_idx_p == frames_preevent-1:
            ax[plotrow,plotcol].patch.set_linewidth(10)
            ax[plotrow,plotcol].patch.set_edgecolor('green')
        ax[plotrow,plotcol].scatter(roi_pos[0]*pxsize+pxsize/2+conf_offset[0], roi_pos[1]*pxsize+pxsize/2+conf_offset[1], marker='x', color='green', s=40)
        ax[plotrow,plotcol].invert_yaxis()
        legend_text = f'Frame {conf_idx_p}'
        if conf_idx == 0:
            legend_text = legend_text + f', post MFX {len(filelist_npy)-conf_idx-1}'
        elif conf_idx < len(filelist_npy)+1:
            legend_text = legend_text + f', pre MFX {len(filelist_npy)-conf_idx}'
        ax[plotrow,plotcol].legend([legend_text], loc='upper right', labelcolor='w', facecolor=None, frameon=False, markerscale=0, prop={'size': 12})
    if len(t) > len(spot_sums):
        t = t[:-1]
    ax[0,0].plot(t, spot_sums, '-', color='k')
    ax[0,0].plot(t, spot_sums_sm, '--', color='gray')
    ax[0,0].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
    ax[0,0].set_xlabel('time (s) (up to event)')
    ax[0,0].set_ylabel('gag intensity, event area (a.u.)')
    ax[0,0].set_xlim([0, t_eventdetect+t_eventdetect/7])
    ax[0,0].set_ylim([0, np.max(spot_sums)])
    ax[0,1].plot(t, spot_sums, '-', color='k')
    ax[0,1].plot(t, spot_sums_sm, '--', color='gray')
    ax[0,1].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
    ax[0,1].set_xlabel('time (s) (full)')
    ax[0,1].set_ylabel('gag intensity, event area (a.u.)')
    ax[0,1].set_ylim([0, np.max(spot_sums)])
    ax[0,2].plot(t, large_sums, '-', color='k')
    ax[0,2].plot(t, large_sums_sm, '--', color='gray')
    ax[0,2].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
    ax[0,2].set_xlabel('time (s) (full)')
    ax[0,2].set_ylabel('gag intensity, larger area (a.u.)')
    ax[0,2].set_ylim([0, np.max(large_sums)])
    ax[0,3].plot(dxs, dys, alpha=0.5, linewidth=1, color='k')
    sz_arr = np.array(dxerrs)*100
    sz_arr[sz_arr==np.inf] = 150
    sc = ax[0,3].scatter(dxs, dys, s=sz_arr, c=mpl.colormaps['jet'](np.linspace(0, 1, len(dxs))), alpha=0.5)
    ax[0,3].set_xlabel('centroid x coord (µm)')
    ax[0,3].set_ylabel('centroid y coord (µm)')
    ax[0,3].set_xlim([-mfx_size/2, mfx_size/2])
    ax[0,3].set_ylim([-mfx_size/2, mfx_size/2])
    ax[0,3].invert_yaxis()
    divider = make_axes_locatable(ax[0,3])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    colormap = plt.cm.get_cmap('jet')
    sm = plt.cm.ScalarMappable(cmap=colormap)
    sm.set_clim(vmin=0, vmax=len(dxs))
    fig.colorbar(sm, cax=cax, orientation='vertical')
    
    # save fig in pdf
    pdf = mpl.backends.backend_pdf.PdfPages(os.path.join(folder,"summary-event.pdf"))
    for i in plt.get_fignums():
        pdf.savefig(i)
    pdf.close()
    plt.close('all')
    plt.show()

C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e1
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e2
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e3
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e4
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e5
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e1
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e2
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e3
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e4
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e5


In [ ]:
## MULTIPLE ROI FOLLOW

%matplotlib agg
%matplotlib agg

eventfolders = eventfolders_all
for folder in eventfolders:
    print(folder)
    filelist = os.listdir(folder)
    filelist_npy_all = [file for file in filelist if file.endswith('.npy') and 'test' not in file]
    roinames = np.unique([file.split('_')[1].split('-')[0] for file in filelist_npy_all])
    for roiname in roinames:
        print(roiname)
        roiid = 'id'+roiname.split('ROI')[1]
        filelist_npy = [file for file in filelist_npy_all if roiname+'-' in file]
        filelist_confs = [file for file in filelist if 'conf' in file and 'analysis' not in file and 'stack' not in file]
        filelist_msr = [file for file in filelist if file.endswith('.msr')]
        filelist_logs = [file for file in filelist if file.endswith('.txt')]

        # load msr file to read confocal image settings
        # get metadata from confocal image in msr file (pixel size, image shape, image size, origin offset)
        file_msr = os.path.join(folder, filelist_msr[0])
        msr_dataset = obf_support.File(file_msr)
        conf_msr_stack_index = 0  # in currently used imspector template file, the confocal dataset is always stack 0 in the .msr file. This might change with other templates used.
        conf_stack = msr_dataset.stacks[conf_msr_stack_index]
        conf_img_last = [np.reshape(conf_stack.data, (conf_stack.shape[0], conf_stack.shape[1])).T]  # read last confocal image from msr file
        pxsize = conf_stack.pixel_sizes[0]*1e6
        pxshift = pxsize/2
        conf_size_px = (conf_stack.shape[0], conf_stack.shape[1])
        conf_size = (conf_stack.lengths[0]*1e6, conf_stack.lengths[1]*1e6)
        conf_offset = (conf_stack.offsets[0]*1e6, conf_stack.offsets[1]*1e6)
        # get roi pos
        roi_pos = (int(filelist_npy[0].split('[')[1].split(',')[0]),int(filelist_npy[0].split(']')[0].split(',')[1]))
        print(roi_pos)
        roi_pos_um = (roi_pos[0]*pxsize+conf_offset[0], roi_pos[1]*pxsize+conf_offset[1])

        # set other parameters for the plotting
        N_rois = len(filelist_msr)
        roi_idx_cum = 0
        nl = '\n'
        plt_roi_size = int(np.ceil(0.8/pxsize))
        spot_sum_size = int(np.ceil(0.24/pxsize))
        mfx_size = 0.8
        large_sum_size = int(np.ceil(mfx_size*1.2/2/pxsize))
        t_finframe = 3  # time in s for final confocal frame preevent and the time between that frame and mfx initialization (should always be 2-5 s, depending on confocal frame size)
        t_mfxtoconf = 9  # time in s between a mfx cycle end and a confocal frame start (+7s due to DM error)

        # get all confocal images in one stack
        images_conf = []
        for conf_file in filelist_confs:
            images_conf.append(tifffile.imread(os.path.join(folder, conf_file)))
        frames_preevent = len(images_conf[0])
        images_conf = np.concatenate(images_conf)
        images_conf = np.concatenate([images_conf, conf_img_last])
        #tifffile.imwrite(file_msr.split('_')[0]+'_confstack.tif', images_conf)

        # read timing info from log file
        file_log = os.path.join(folder, filelist_logs[0])
        log_dict = {}
        with open(file_log) as f:
            for line in f:
                key, sep, value = line.strip().partition(": ")
                log_dict[key] = value
        t_mfxcycs = []
        try:
            t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
            t_mfxcyc0 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
            t_mfxcyc1 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
            t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between ROI acquisitions during MFX
        except:
            try:
                t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
                t_mfxcyc0 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
                t_mfxcyc1 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle2'], '%Hh%Mm%Ss%fus')
                t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between ROI acquisitions during MFX
            except:
                t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
                t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
                t_sec_mfx = (t_mfxcyc0-t_confevent_end).seconds  # time between ROI acquisitions during MFX
        try:
            t_conf0_start = datetime.datetime.strptime(log_dict['experiment_start'], '%Hh%Mm%Ss%fus')
        except:
            t_conf0_start = datetime.datetime.strptime('12h00m00s000000us', '%Hh%Mm%Ss%fus')
        t_sec_frames_preevent = (t_confevent_end-t_conf0_start).seconds  # time between confocal frames before MFX
        t_sec_mfx0 = (t_mfxcyc0-t_confevent_end).seconds  # time for one MFX rec

        # get t array for all confocal frames
        t = np.linspace(0, t_sec_frames_preevent-t_finframe, frames_preevent)
        t_eventdetect = t[-1]
        t = np.append(t, t[-1]+t_sec_mfx0+t_mfxtoconf)
        for _ in range(len(filelist_confs)-2+1):
            t = np.append(t,t[-1]+t_sec_mfx+t_mfxtoconf)

        # create figure for plotting
        cols = 4
        rows = 1+(len(images_conf)//cols+1)  # 1 for plots, the rest for the number of confocal frames
        fig, ax = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
        # get intensity limits for confocal images
        conf_zoom_lim = (np.min(images_conf[:,roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size]), np.max(images_conf[:,roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size]))
        # get x,y lims for confocal image zooms
        lims_confimg = np.array([roi_pos_um[0], roi_pos_um[0], roi_pos_um[1], roi_pos_um[1]]) + np.array([-plt_roi_size, plt_roi_size, -plt_roi_size, plt_roi_size])*pxsize
        # plot image zooms and plots of intensity over time
        spot_sums = []
        large_sums = []
        dxs = []
        dys = []
        dxerrs = []
        dyerrs = []
        for conf_idx_p in range(len(images_conf)):
            # summing
            conf_idx = len(images_conf)-conf_idx_p-1
            plotrow = conf_idx//cols+1
            plotcol = conf_idx%cols
            conf_img = images_conf[conf_idx_p]
            conf_img_spot_cut = conf_img[roi_pos[1]-spot_sum_size:roi_pos[1]+spot_sum_size+1, roi_pos[0]-spot_sum_size:roi_pos[0]+spot_sum_size+1]
            conf_img_large_cut = conf_img[roi_pos[1]-large_sum_size:roi_pos[1]+large_sum_size+1, roi_pos[0]-large_sum_size:roi_pos[0]+large_sum_size+1]
            spot_sums.append(np.sum(np.sum(conf_img_spot_cut))/spot_sum_size**2)
            large_sums.append(np.sum(np.sum(conf_img_large_cut))/large_sum_size**2)
            rolling_window = 9
            spot_sums_sm = uniform_filter1d(spot_sums, size=rolling_window, mode='nearest')
            large_sums_sm = uniform_filter1d(large_sums, size=rolling_window, mode='nearest')
            # fitting centroid of spot with symmetric 2D gaussian
            x = np.linspace(0, 2*(large_sum_size+1)-2, 2*(large_sum_size+1)-1)
            y = np.linspace(0, 2*(large_sum_size+1)-2, 2*(large_sum_size+1)-1)
            x, y = np.meshgrid(x, y)
            try:
                initial_guess = (np.max(conf_img_large_cut), large_sum_size, large_sum_size, 1, 0, 10)
                popt, pcov = curve_fit(gaussian2D, (x, y), conf_img_large_cut.ravel(), p0=initial_guess)
                # get offsets from pixel-position
                dx = (popt[1]-large_sum_size)*pxsize
                dy = (popt[2]-large_sum_size)*pxsize
                x0err = np.sqrt(np.diag(pcov))[2]
                y0err = np.sqrt(np.diag(pcov))[1]
            except:
                dx = np.nan
                dy = np.nan
                x0err = np.nan
                y0err = np.nan
            dxs.append(dx)
            dys.append(dy)
            dxerrs.append(x0err)
            dyerrs.append(y0err)
            # plotting
            img_onlyconf = ax[plotrow,plotcol].imshow(conf_img, cmap='hot', vmin=conf_zoom_lim[0], vmax=conf_zoom_lim[1])
            if conf_idx_p == 0:
                extents_confimg = np.array(img_onlyconf.get_extent())*pxsize+[pxsize/2, pxsize/2, pxsize/2, pxsize/2]+[conf_offset[0], conf_offset[0], conf_offset[1], conf_offset[1]]
            img_onlyconf.set_extent(extents_confimg)  # scale overlay image to the correct pixel size for the tracks
            ax[plotrow,plotcol].set_xlim(lims_confimg[:2])
            ax[plotrow,plotcol].set_ylim(lims_confimg[2:])
            if conf_idx_p == frames_preevent-1:
                ax[plotrow,plotcol].patch.set_linewidth(10)
                ax[plotrow,plotcol].patch.set_edgecolor('green')
            ax[plotrow,plotcol].scatter(roi_pos[0]*pxsize+pxsize/2+conf_offset[0], roi_pos[1]*pxsize+pxsize/2+conf_offset[1], marker='x', color='green', s=40)
            ax[plotrow,plotcol].invert_yaxis()
            legend_text = f'Frame {conf_idx_p}'
            if conf_idx == 0:
                legend_text = legend_text + f', post MFX {len(filelist_npy)-conf_idx-1}'
            elif conf_idx < len(filelist_npy)+1:
                legend_text = legend_text + f', pre MFX {len(filelist_npy)-conf_idx}'
            ax[plotrow,plotcol].legend([legend_text], loc='upper right', labelcolor='w', facecolor=None, frameon=False, markerscale=0, prop={'size': 12})
        if len(t) > len(spot_sums):
            t = t[:-1]
        ax[0,0].plot(t, spot_sums, '-', color='k')
        ax[0,0].plot(t, spot_sums_sm, '--', color='gray')
        ax[0,0].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
        ax[0,0].set_xlabel('time (s) (up to event)')
        ax[0,0].set_ylabel('gag intensity, event area (a.u.)')
        ax[0,0].set_xlim([0, t_eventdetect+t_eventdetect/7])
        ax[0,0].set_ylim([0, np.max(spot_sums)])
        ax[0,1].plot(t, spot_sums, '-', color='k')
        ax[0,1].plot(t, spot_sums_sm, '--', color='gray')
        ax[0,1].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
        ax[0,1].set_xlabel('time (s) (full)')
        ax[0,1].set_ylabel('gag intensity, event area (a.u.)')
        ax[0,1].set_ylim([0, np.max(spot_sums)])
        ax[0,2].plot(t, large_sums, '-', color='k')
        ax[0,2].plot(t, large_sums_sm, '--', color='gray')
        ax[0,2].axvline(x=t_eventdetect, linewidth=8, color='g', alpha=0.6)
        ax[0,2].set_xlabel('time (s) (full)')
        ax[0,2].set_ylabel('gag intensity, larger area (a.u.)')
        ax[0,2].set_ylim([0, np.max(large_sums)])
        ax[0,3].plot(dxs, dys, alpha=0.5, linewidth=1, color='k')
        sz_arr = np.array(dxerrs)*100
        sz_arr[sz_arr==np.inf] = 150
        sc = ax[0,3].scatter(dxs, dys, s=sz_arr, c=mpl.colormaps['jet'](np.linspace(0, 1, len(dxs))), alpha=0.5)
        ax[0,3].set_xlabel('centroid x coord (µm)')
        ax[0,3].set_ylabel('centroid y coord (µm)')
        ax[0,3].set_xlim([-mfx_size/2, mfx_size/2])
        ax[0,3].set_ylim([-mfx_size/2, mfx_size/2])
        ax[0,3].invert_yaxis()
        divider = make_axes_locatable(ax[0,3])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        colormap = plt.cm.get_cmap('jet')
        sm = plt.cm.ScalarMappable(cmap=colormap)
        sm.set_clim(vmin=0, vmax=len(dxs))
        fig.colorbar(sm, cax=cax, orientation='vertical')
        
        # save fig in pdf
        pdf = mpl.backends.backend_pdf.PdfPages(os.path.join(folder,f"summary-event-{roiname}.pdf"))
        for i in plt.get_fignums():
            pdf.savefig(i)
        pdf.close()
        plt.close('all')
        plt.show()

C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\250123\sample1\e1
ROI0
(132, 145)
ROI1
(155, 120)
ROI2
(75, 129)
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\250123\sample1\e2
ROI0
(145, 49)
ROI1
(73, 55)
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\250123\sample2\e2
ROI2
(115, 31)
ROI3
(127, 97)
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\250123\sample3\e2
ROI3
(156, 22)
ROI4
(63, 65)


In [ ]:
### Print all ROI names, respective number of ROI cycles, and respective MFX period
for folder in eventfolders_all:
    print(folder)
    filelist = os.listdir(folder)
    filelist_npy_all = [file for file in filelist if file.endswith('.npy') and 'test' not in file]
    roinames = np.unique([file.split('_')[1].split('-')[0] for file in filelist_npy_all])
    filelist_msr = [file for file in filelist if file.endswith('.msr')]
    filelist_logs = [file for file in filelist if file.endswith('.txt')]
    filelist_confs = [file for file in filelist if 'conf' in file and 'analysis' not in file and 'stack' not in file]
    for roiname in roinames:
        print(roiname)
        roiid = 'id'+roiname.split('ROI')[1]
        filelist_npy = [file for file in filelist_npy_all if roiname+'-' in file]
        print(len(filelist_npy))
        
        # read timing info from log file
        file_log = os.path.join(folder, filelist_logs[0])
        log_dict = {}
        with open(file_log) as f:
            for line in f:
                key, sep, value = line.strip().partition(": ")
                log_dict[key] = value
        t_mfxcycs = []
        if folder.split('\\')[8] in ['241016','241105','241114','241115','241212']:
            try:
                t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate-cycle0'], '%Hh%Mm%Ss%fus')
                t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end-cycle0'], '%Hh%Mm%Ss%fus')
                try:
                    t_mfxcyc1 = datetime.datetime.strptime(log_dict['mfx_end-cycle1'], '%Hh%Mm%Ss%fus')
                except:
                    t_mfxcyc1 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
                t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between confocal frames during MFX
            except:
                try:
                    t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate-cycle0'], '%Hh%Mm%Ss%fus')
                except:
                    t_confevent_end = datetime.datetime.strptime(log_dict['mfx_initiate'], '%Hh%Mm%Ss%fus')
                t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
                t_sec_mfx = (t_mfxcyc0-t_confevent_end).seconds  # time between confocal frames during MFX
            try:
                t_conf0_start = datetime.datetime.strptime(log_dict['experiment_start'], '%Hh%Mm%Ss%fus')
            except:
                t_conf0_start = datetime.datetime.strptime('12h00m00s000000us', '%Hh%Mm%Ss%fus')
            t_sec_frames_preevent = (t_confevent_end-t_conf0_start).seconds  # time between confocal frames before MFX
            t_sec_mfx0 = (t_mfxcyc0-t_confevent_end).seconds  # time for one MFX rec
        else:
            try:
                t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
                t_mfxcyc0 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
                t_mfxcyc1 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
                t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between ROI acquisitions during MFX
            except:
                try:
                    t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
                    t_mfxcyc0 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle1'], '%Hh%Mm%Ss%fus')
                    t_mfxcyc1 = datetime.datetime.strptime(log_dict[f'mfx_end-{roiid}-cycle2'], '%Hh%Mm%Ss%fus')
                    t_sec_mfx = (t_mfxcyc1-t_mfxcyc0).seconds  # time between ROI acquisitions during MFX
                except:
                    t_confevent_end = datetime.datetime.strptime(log_dict[f'mfx_initiate-{roiid}-cycle0'], '%Hh%Mm%Ss%fus')
                    t_mfxcyc0 = datetime.datetime.strptime(log_dict['mfx_end'], '%Hh%Mm%Ss%fus')
                    t_sec_mfx = (t_mfxcyc0-t_confevent_end).seconds  # time between ROI acquisitions during MFX
            try:
                t_conf0_start = datetime.datetime.strptime(log_dict['experiment_start'], '%Hh%Mm%Ss%fus')
            except:
                t_conf0_start = datetime.datetime.strptime('12h00m00s000000us', '%Hh%Mm%Ss%fus')
            t_sec_frames_preevent = (t_confevent_end-t_conf0_start).seconds  # time between confocal frames before MFX
            t_sec_mfx0 = (t_mfxcyc0-t_confevent_end).seconds  # time for one MFX rec
        print(t_sec_mfx)

C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e1
ROI0
8
31
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e2
ROI0
6
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e3
ROI0
8
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e4
ROI0
9
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample2\e5
ROI0
50
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e1
ROI0
6
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e2
ROI0
2
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e3
ROI0
8
41
C:\Users\alvelidjonatan\Documents\Data\etMINFLUX-lab\data-gag\tracking3d-srmemb\241016\sample3\e4
ROI0
7
41
C:\Users\alvelidjonatan\Doc